## **Segmentation of mitochondria in WebKnossos datasets with Empanada**
---
Perform batch segmentation of mitochondria using MitoNet for a full data set
- Import dataset in chunks from WebKnossos
- Perform 3D inference of mitochondria using `Empanada`
- Upload resulting segmentations as a segmentation layer to WebKnossos

In [1]:
# Data import
import webknossos as wk
from mitonet_seg.utils import import_wk_dataset_local

# 3D inference
from mitonet_seg.utils import inference_3d

# Empanada
import os
from empanada.inference.patterns import *

### **3.1 Configure dataset parameters**
---

In [10]:
TOKEN = "A_KRh0eDGiIuQMTL1EKxWg" # Generate from https://webknossos.tnw.tudelft.nl/auth/token. Don't share this token online or in publications!!!
url = "https://webknossos.tnw.tudelft.nl" # "https://webknossos.tnw.tudelft.nl" 
dataset_name = "20231107_MC7_UAC_realigned_full_1x" # Dataset name as in WebKnossos
organization_id = "hoogenboom-group" # "hoogenboom-group"
dir_path = f"/long_term_storage/webknossos/binaryData/hoogenboom-group/{dataset_name}"
config = os.path.abspath("configs/MitoNet_v1.yaml") # MitoNet model configuration file
layer = "postcorrection_realigned_SOFIMA"

mag_x, mag_y, mag_z = 4, 4, 1 # Magnification level (x, y, z) in WebKnossos to be used for segmentation. Default is (4, 4, 1)
bbox_size = 2048 # pixels, at desired zoom level. Default is 2048
MAG = wk.Mag(f"{mag_x}-{mag_y}-{mag_z}") # Set magnification for WebKnossos

### **3.2 (Remote) import data from WebKnossos**
---

In [11]:
# # Import full data set at required zoom level (remote)
# mag_view, voxelsize = import_wk_dataset_local(TOKEN, 
#                                                url, 
#                                                dataset_name, 
#                                                organization_id, 
#                                                MAG)

# Import full data set at required zoom level (local)
dataset, mag_view, voxelsize = import_wk_dataset_local(dir_path,
                                                       MAG,
                                                       layer=layer)

In [12]:
EM = dataset.get_layer(layer) # EM data Layer
mag_view = EM.get_mag(MAG) # MagView
data = mag_view.read()
data.shape

(1, 8192, 8192, 54)

In [13]:
EM.bounding_box

BoundingBox(topleft=(28056, 27286, 0), size=(32766, 32766, 54))

### **3.3 Run Empanada (MitoNet)**
---
Reads data into Napari and performs the segmentation with Empanada

In [14]:
from tqdm.notebook import tqdm

In [15]:
# Make new "segmentation" layer
segmentation_layer = dataset.add_layer(
        "realigned_SOFIMA_MitoNet", 
        wk.SEGMENTATION_CATEGORY,
        compressed=True,
        dtype_per_layer="uint32",
        largest_segment_id=None)
segmentation_layer.bounding_box = EM.bounding_box.align_with_mag(MAG)
mag = segmentation_layer.add_mag(MAG, compress=True)


In [16]:
mag

MagView(name='4-4-1', bounding_box=BoundingBox(topleft=(28056, 27288, 0), size=(32764, 32764, 54)))

In [19]:
# Run 3d inference (weirdly has to be on yz stack)
mito_labels = inference_3d(config, data, mode='stack', qlen=1, nmax=20000, seg_thr=0.5, nms_thr=0.1, nms_kernel=3, 
                           iou_thr=0.25, ioa_thr=0.25, pixel_vote_thr=2, cluster_io_thr=0.75, min_size=200, 
                           min_span=1, downsample_f=1, one_view=True, fine_boundaries=False, use_cpu=True, nworkers=10)    

/home/akievits/mitonet-seg/.venv/lib/python3.9/site-packages/torch/serialization.py:1007: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


Predicting yz stack


  0%|          | 0/54 [01:23<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Add "Mag" and write segmentation data to "Mag"
mag.write(data=mito_labels,
          absolute_offset=segmentation_layer.bounding_box.topleft)
segmentation_layer.refresh_largest_segment_id()

# Create other zoom levels
print("Downsampling resulting segmentations...")
segmentation_layer.downsample(sampling_mode="constant_z")
print("Downsampling completed...")